### Import Dependencies

In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
import torch

torch.cuda.is_available()

True

### Load ENV

In [3]:
enviorment_name = "CartPole-v1"
env = gym.make(enviorment_name, render_mode = 'human')

In [4]:
episodes= 5 
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    terminated = False
    truncated = False

    while not (terminated or truncated):
        env.render()
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        score += reward
    print(f"Episode:{episode} Score:{score}")
    
env.close()

/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Episode:1 Score:10.0
Episode:2 Score:27.0
Episode:3 Score:14.0
Episode:4 Score:19.0
Episode:5 Score:47.0


### Train an RL Model

In [8]:
# Make the directory for saving the logs

log_path = os.path.join('..','Training', 'Logs')
os.makedirs(log_path, exist_ok=True)

In [9]:
log_path

'../Training/Logs'

In [6]:
env = gym.make(enviorment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [7]:
model.learn(total_timesteps=20000)

Logging to ../Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 1203 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 977         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009232572 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00361    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.94        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0172     |
|    value_loss           | 51.5        |
-----------------------------------------


### Save and Reload Model

In [8]:
PPO_Path = os.path.join('..','Training', 'Saved Models', 'PPO_CartPole_Model')
os.makedirs(os.path.dirname(PPO_Path), exist_ok=True)

In [9]:
model.save(PPO_Path)


In [10]:
del model 

In [11]:
model = PPO.load(PPO_Path, env=env)

/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


### Evaluation

In [12]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:259: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(np.float64(500.0), np.float64(0.0))

### Test Model

In [35]:
model, _ = model.predict(state)
model

array([0])

In [13]:
episodes= 5 
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(obs) # Using the trained model to predict the action
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode:{episode} Score:{score}")
    
env.close()

Episode:1 Score:[500.]
Episode:2 Score:[500.]
Episode:3 Score:[500.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


### Viewing logs in Tensorboard

In [14]:
training_logs = os.path.join(log_path, 'PPO_2')

In [15]:
training_logs

'../Training/Logs/PPO_2'

In [16]:
!tensorboard --logdir {training_logs}

/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/tensorboard/default.py:30: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.20.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


### Adding a callback to the training Stage

In [4]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [5]:
save_path = os.path.join('..','Training', 'Saved Models')
save_path

'../Training/Saved Models'

In [6]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path= save_path,
                             verbose=1)

In [7]:
PPO = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

NameError: name 'log_path' is not defined

In [26]:
model.learn(total_timesteps=200000, callback=eval_callback)

Logging to ../Training/Logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 1258 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1005        |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004258439 |
|    clip_fraction        | 0.0303      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.544      |
|    explained_variance   | -0.000376   |
|    learning_rate        | 0.0003      |
|    loss                 | 0.444       |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00265    |
|    value_loss           | 25.1        |
-----------------------------------------


/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 500          |
|    mean_reward          | 500          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0036898663 |
|    clip_fraction        | 0.0177       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.536       |
|    explained_variance   | 0.0119       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.19         |
|    n_updates            | 140          |
|    policy_gradient_loss | -0.00274     |
|    value_loss           | 2.32         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 500.00  is above the threshold 200


### Changing Policies

In [10]:
net_arch = [dict(pi =[128, 128,128,128], vf =[128, 128,128,128])]

In [11]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(
/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than

In [12]:
model.learn(total_timesteps=200000, callback=eval_callback)

/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Logging to ../Training/Logs/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.3     |
|    ep_rew_mean     | 21.3     |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 43       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.8        |
|    ep_rew_mean          | 26.8        |
| time/                   |             |
|    fps                  | 47          |
|    iterations           | 2           |
|    time_elapsed         | 86          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013530438 |
|    clip_fraction        | 0.154       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00348  

/home/igrise/Project Coding/Reinforcement Learning Projects/gymnasium_learn/.venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 500         |
|    mean_reward          | 500         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.012591569 |
|    clip_fraction        | 0.12        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.575      |
|    explained_variance   | 0.537       |
|    learning_rate        | 0.0003      |
|    loss                 | 4.61        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0169     |
|    value_loss           | 34.8        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 500.00  is above the threshold 200


### Using an Alternate Algorithm

In [13]:
from stable_baselines3 import DQN 

In [14]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=20000)